In [2]:
## Import packages

import numpy as np
import matplotlib.pyplot as plt
import copy

In [3]:
## Set parameters

# utility functions are a * x^0.5 + b
a_vect = [1,1,1,1,1,1,1,1,1,1]
b_vect = [0,0,0,0,0,0,0,0,0,0]

K = len(a_vect)

constr_A = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
C = np.array([1,1,1,1,1,1,1,1,1,1])

type_obj = 0 # 0 is sqrt root, 1 is log

stop_iter = 1000
delta_x = 0.01
delta_l = 0.01

In [7]:
## Functions of objective and constraint

def sqrt_func(a, b, x):
    return a * x**0.5 + b

def der_sqrt_func(a, x):
    if x == 0:
        return 0.5 * a / (0.0001**0.5)
    return 0.5 * a / (x**0.5)

def log_func(a, b, x):
    return log(np.dot(a, x) + 1) + b

def der_log_func(a, x):
    return a / (np.dot(a, x) + 1) + b
    
def lin_constr(A, C, x):
    return C- np.matmul(A,x)

local_objs = []
for a,b in zip(a_vect, b_vect):
    if type_obj == 0:
        local_objs.append(lambda x, a=a, b=b: sqrt_func(a,b,x))
    else:
        local_objs.append(lambda x, a=a, b=b: log_func(a,b,x))
        
local_ders = []
for a,b in zip(a_vect, b_vect):
    if type_obj == 0:
        local_ders.append(lambda x, a=a: der_sqrt_func(a,x))
    else:
        local_ders.append(lambda x, a=a: der_log_func(a,x))
    
def obj_func(x):
    s = 0
    for (x_i, f) in zip(x, local_objs):
        s += f(x_i)
    return s

def obj_func_der(x):
    out = np.zeros((K,))
    for i in range(K):
        out[i] = local_ders[i](x[i])
    return out
    
constr = (lambda x, A=constr_A, C=C: lin_constr(A, C, x))